In [1]:
import requests
from elasticsearch import Elasticsearch
import json
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search,Q
from credentials import usr_pwd

In [2]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from nltk.tokenize import sent_tokenize
import numpy as np
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

/home/pavan_rs/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


TF version:  2.9.1
Hub version:  0.12.0


### update ES document mappings with dense vectors

In [3]:
hosts = ["10.1.8.6:9200"]
uname, pwd =  usr_pwd()

In [4]:
es = Elasticsearch(hosts,
                  timeout = 6000,
                  use_ssl = True,
                  verify_certs = False,
                  ssl_show_warn = False,
                  raise_on_error = False,
                  http_auth = (uname,pwd)
                  )

In [5]:
es.ping()

True

In [6]:
res_cnt = es.count(index='test_cantire-2021-10', body={'query':{"match_all":{}} })["count"]
# res_cnt = es.count(index='test_cantire-2021-10', body=body)["count"]
res_cnt

945973

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L12-v2')

In [8]:
from nltk.tokenize import sent_tokenize

In [9]:
def get_elastic_results(es):
    
    res = es.search(index = "test_cantire-2021-10",body = {
    "size": 500,
    "query": {
    "bool": {
#     "filter":{
    "must_not": [{
    "exists": {
    "field": "doc_vectors_n"
    }
    },
#     }
#       {
#           "exists":{
#               "field":"status"
#           }
#       }  
    ],
    "must": [
    {
    "match_phrase": {
    "fileType": "Message/Email File"
    }
    }
    ]
    }
    }
    })
    
    return res

In [39]:
# PUT /test_cantire-2021-10/_mapping
# {
#   "mappings": {
#     "properties": {
#       "doc_vectors_n": {
#         "type": "nested",
#         "properties": {
#           "vector": {
#             "type": "dense_vector",
#             "dims": 384  
#           }
#         }
#       }
#     }
#   }
# }

In [10]:
g = get_elastic_results(es)
# g["hits"]["hits"][0]
print(len(g["hits"]["hits"]))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


500


In [12]:
# print(g["hits"]["hits"][4]["_source"]["content"])

In [11]:
from tqdm import tqdm
def update_es_vector(es, es_res):
    all_documents = es_res['hits']['hits']
    print(len(all_documents))
    if len(all_documents) > 0:
        for doc in tqdm(all_documents):
            sent_embedding = []
            email_content = doc['_source']['content']
            email_content = email_content.split('From')[0].split('__')[0]
            if email_content != None:
                email_content = (' '.join(''.join(email_content.split('\n')).split('\r')).lower())
                email_content = ' '.join(email_content.split('\t'))
                doc_id = doc['_id']
                _docum_ = "https://search.discoverdollar.com:9200/test_cantire-2021-10/"+ doc_id
                sent_lis = sent_tokenize(email_content)
                for i in sent_lis:
                    sent_embedding.append((np.array(list(model.encode([i]).flatten()))).tolist())
                doc_vecs_1 = [{'vector' : emb} for emb in sent_embedding]

                try:
                    es.update(index = 'test_cantire-2021-10', doc_type = "_doc", id=doc_id, body={"doc":{"doc_vectors_n":doc_vecs_1}})
                    es.update(index = 'test_cantire-2021-10', doc_type = "_doc", id=doc_id, body={"doc":{"status":"yes"}})
                except:
                    print("an exception occurred")

In [12]:
# es_res = get_elastic_results(es)
# update_es_vector(es, es_res)

In [13]:
cnt = res_cnt//1000
cnt

945

In [ ]:
i=0
for i in range(50):
    i+=1
    es_res = get_elastic_results(es)
    if len(es_res["hits"]["hits"]) > 0:
        update_es_vector(es, es_res)
    else:
        break
    print(i)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  0%|          | 0/500 [00:00<?, ?it/s]

500


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'body' parameter is deprecated for the 'update' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The 'body' parameter is deprecated for the 'update' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
100%|██████████| 500/500 [04:38<00:00,  1.80it/s]


1


  0%|          | 0/500 [00:00<?, ?it/s]

...

100%|██████████| 500/500 [04:45<00:00,  1.75it/s]


13


  0%|          | 0/500 [00:00<?, ?it/s]

500


 23%|██▎       | 113/500 [01:03<03:27,  1.87it/s]

In [14]:
body = {
    "query":{
        "bool":{
            "filter":{
                "exists":{
                    "field":"status"
                }
            },
            "must":[{"match_phrase_prefix":{"fileType":"Email"}},{"match_phrase":{"status":"yes"}}]
        }
    }
}
result = es.search(index = "test_cantire-2021-10",body=body)
print(len(result["hits"]["hits"]))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  del sys.path[0]


10


In [15]:
es.count(index='test_cantire-2021-10', body=body)["count"]

72837

In [17]:
len(result["hits"]["hits"][0]["_source"]["doc_vectors_n"])
# result["hits"]["hits"][30]["_source"]
# result["hits"]["hits"][0]["_source"]["doc_vectors2"]

5

### Build API to call queries

In [18]:
import numpy as np

In [19]:
query = ['sku cost update']
query_vec = model.encode(query)
query_vect = np.array(list(query_vec.flatten()))
query_vector = query_vect.tolist()
len(query_vector)

384

In [50]:
# s_body = {
#     "query":{
#         "score_mode":"max",
#         "script_score": {
#             "query": {
#                 "bool":{
#                     "filter":{
#                         "exists":{
#                             "field":"marked1"
#                         }
#                     }
#                 }
#             },
#             "script": {
#                 "source": "cosineSimilarity(params.query_vector, 'doc_vectors5.vector') + 1.0",
#                 "params": {"query_vector": query_vector}
#                 }
#             }
#         }
#     }

In [20]:
s_body = {
  "min_score":1.5,
  "query": {
    "nested": {
      "path": "doc_vectors_n",
        "score_mode": "max", 
      "query": {
          "function_score": {
              "script_score": {
                  "script": {
                      "source": "(1.0+cosineSimilarity(params.query_vector, 'doc_vectors_n.vector'))",
                      "params": {"query_vector": query_vector}
                      }
                  }
              }
          }
        }
      }
    }

In [33]:
# result_vectors = es.search(index="test_cantire-2021-10", body=s_body, size=100)

In [34]:
# result_vectors["hits"]["hits"][0]

In [21]:
vec_res = es.search(index="test_cantire-2021-10",body=s_body,size=1000)
def clean_mail(mail_):
    email_content = mail_['_source']['content']

    email_content = email_content.split('From')[0].split('__')[0]
    if not (email_content or email_content == None):

        email_content = (' '.join(''.join(email_content.split('\n')).split('\r')).lower())
        email_content = ' '.join(email_content.split('\t'))
        
    return email_content

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.


In [22]:
len(vec_res["hits"]["hits"])

233

In [23]:
for vectored_doc in vec_res['hits']['hits'][:10]:
    print('=============', vectored_doc['_index'], '==============')
    print(clean_mail(vectored_doc))

============= test_cantire-2021-10 ==============
Dear Annie,

Herewith the PO forecast for your reference, to ensure Feb shipment PO 
in correct price, pls kindly release RFX asap.

Thank you~

Best regards,
Cherry Chow
Tiffany Materials, Ltd.
Unit A, 11/F, YHC Tower,
No.1 Sheung Yuet Road,
Kowloon Bay, Kowloon,  Hong Kong
Tel.852-2722 2617
Fax:852-2721 9692
Website: www.tiffanymaterials.com
************************************************************

On 30/9/2021 9:52, Cherry Chow wrote:
> Dear Annie,
>
> Double checked with our top management, it is a MUST to ask for price 
> increase for the PO which placed after 1OCT2021, pls kindly release 
> RFX in which allow us to update the PO price.
>
> thanks
>
> Best regards,
> Cherry Chow
> Tiffany Materials, Ltd.
> Unit A, 11/F, YHC Tower,
> No.1 Sheung Yuet Road,
> Kowloon Bay, Kowloon,  Hong Kong
> Tel.852-2722 2617
> Fax:852-2721 9692
> Website:www.tiffanymaterials.com
> ************************************************************
> 

In [56]:
from flask import Flask

In [57]:
app = Flask(__name__)

@app.route('/')


def search_body(model=embed):
    if request.method == 'POST':
        query = request.form.to_dict()
        query=list(to_predict_list.values())
        query_vector = np.array(model(query))
        s_body = {
          "query": {
            "nested": {
              "path": "doc_vectors",
              "score_mode": "max", 
              "query": {
                "function_score": {
                  "script_score": {
                    "script": {
                      "source": "(1.0+cosineSimilarity(params.query_vector, 'doc_vectors.vector'))",
                      "params": {"query_vector": query_vector[0]}
                    }
                  }
                }
              }
            }
          }
        }
        

    return s_body

s_body = search_body(embed)

def get_vector_documents(es_host= es_set, es_index = index, body= s_body ):
        result_vectors = es_set.search(index="cantire-test-2020-06", body=s_body, size = 100)
        for vectored_doc in result_vectors['hits']['hits']:
            print('=============', vectored_doc['_index'], '==============')
            print(clean_mail(vectored_doc))


NameError: name 'request' is not defined

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)